<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/code/kogpt_%EA%B8%B0%EB%B0%98_%EC%A6%9D%EB%B6%84_0_5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━ 120.0/207.5 MB 194.1 MB/s eta 0:00:01

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 및 토크나이저 로드
MODEL_NAME = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

# PAD 토큰 설정
tokenizer.pad_token_id = tokenizer.eos_token_id

# 5개의 프롬프트 템플릿 (각각 변수로 저장)
prompt1 = "다음 문장을 다른 표현으로 바꿔 주세요: {}"
prompt2 = "비슷한 내용을 가진 새로운 문장을 만들어 주세요: {}"
prompt3 = "주어진 문장을 좀 더 자연스럽고 상세하게 만들어 주세요: {}"
prompt4 = "다음 문장을 20자 이내로 요약해 주세요: {}"
prompt5 = "다음 문장을 100자 이상으로 확장해 주세요: {}"

# 데이터 증강 함수 정의
def generate_augmented_texts(prompt, max_length=100):
    # 5개의 프롬프트 각각 적용
    formatted_prompts = [
        prompt1.format(prompt),
        prompt2.format(prompt),
        prompt3.format(prompt),
        prompt4.format(prompt),
        prompt5.format(prompt)
    ]

    generated_texts = []

    for final_prompt in formatted_prompts:
        encoding = tokenizer(final_prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        input_ids = encoding.input_ids.to(model.device)
        attention_mask = encoding.attention_mask.to(model.device)

        with torch.no_grad():
            output = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_new_tokens=30,
                temperature=0.5,
                top_p=0.8,
                do_sample=True,
                early_stopping=True,
                repetition_penalty=1.2
            )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_texts.append(generated_text)

    # 각각 변수로 저장
    generated_text1, generated_text2, generated_text3, generated_text4, generated_text5 = generated_texts

    return generated_text1, generated_text2, generated_text3, generated_text4, generated_text5

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [9]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/My Drive/졸업논문/DATA/NSMC_0.5_증분대상_2500개.csv', encoding='utf-8-sig')

In [10]:
df.head(3)

,id,document,label,길이
0,7652286,솔직히 이영화때문에 슈퍼맨시리즈가 7년이란 세월동안 묻혀있엇다 맨오브스틸은 미국에서...,0,138
1,8895340,아무리 짱짱맨 옵대장의 다때려부수는 CG가 좋다그래도 이딴 디워랑 견줄만한 스토리로...,0,101
2,6549317,오프닝장면 죠스영화 고대루 베겼네?? 여자 비명 연기하는 것도 고대루네~! 그냥 갖...,0,103


In [11]:
# 데이터프레임에 적용
df[['prompt1', 'prompt2', 'prompt3', 'prompt4', 'prompt5']] = df['document'].apply(lambda x: pd.Series(generate_augmented_texts(x, max_length=100)))

# 결과 확인

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [12]:
df.to_csv('kogpt증분완료_0.5.csv', encoding='utf-8-sig')

In [13]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 및 토크나이저 로드
MODEL_NAME = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

# PAD 토큰 설정
tokenizer.pad_token_id = tokenizer.eos_token_id

# 5개의 프롬프트 템플릿
prompt1 = "다음 문장을 다른 표현으로 바꿔 주세요: {}"
prompt2 = "비슷한 내용을 가진 새로운 문장을 만들어 주세요: {}"
prompt3 = "주어진 문장을 좀 더 자연스럽고 상세하게 만들어 주세요: {}"
prompt4 = "다음 문장을 20자 이내로 요약해 주세요: {}"
prompt5 = "다음 문장을 100자 이상으로 확장해 주세요: {}"

# 데이터 증강 함수 정의
def generate_augmented_texts(prompt, num_samples=2):
    """ 한 문장에 대해 5개의 프롬프트를 적용하고, 각 프롬프트당 num_samples개의 문장을 생성 """
    formatted_prompts = [
        prompt1.format(prompt),
        prompt2.format(prompt),
        prompt3.format(prompt),
        prompt4.format(prompt),
        prompt5.format(prompt)
    ]

    generated_texts = []

    for final_prompt in formatted_prompts:
        encoding = tokenizer(final_prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = encoding.input_ids.to(model.device)
        attention_mask = encoding.attention_mask.to(model.device)

        for _ in range(num_samples):  # 각 프롬프트당 num_samples개의 문장 생성
            with torch.no_grad():
                output = model.generate(
                    input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=30,  # `max_length` 대신 `max_new_tokens`
                    temperature=0.5,
                    top_p=0.8,
                    do_sample=True,
                    repetition_penalty=1.2
                )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
            generated_texts.append(generated_text)

    return generated_texts  # 10개의 문장 리스트 반환

In [14]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/My Drive/졸업논문/DATA/NSMC_0.1_증분대상_500개.csv', encoding='utf-8-sig')

In [15]:
df[['aug1', 'aug2', 'aug3', 'aug4', 'aug5', 'aug6', 'aug7', 'aug8', 'aug9', 'aug10']] = df['document'].apply(lambda x: pd.Series(generate_augmented_texts(x, num_samples=2)))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [16]:
df.to_csv('/content/gdrive/MyDrive/My Drive/졸업논문/kogpt증분완료_0.1.csv', encoding='utf-8-sig')

In [17]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 및 토크나이저 로드
MODEL_NAME = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

# PAD 토큰 설정
tokenizer.pad_token_id = tokenizer.eos_token_id

# 5개의 프롬프트 템플릿
prompt1 = "다음 문장을 다른 표현으로 바꿔 주세요: {}"
prompt2 = "비슷한 내용을 가진 새로운 문장을 만들어 주세요: {}"
prompt3 = "주어진 문장을 좀 더 자연스럽고 상세하게 만들어 주세요: {}"
prompt4 = "다음 문장을 20자 이내로 요약해 주세요: {}"
prompt5 = "다음 문장을 100자 이상으로 확장해 주세요: {}"

# 데이터 증강 함수 정의
def generate_augmented_texts(prompt, num_samples=10):
    """ 한 문장에 대해 5개의 프롬프트를 적용하고, 각 프롬프트당 num_samples개의 문장을 생성 """
    formatted_prompts = [
        prompt1.format(prompt),
        prompt2.format(prompt),
        prompt3.format(prompt),
        prompt4.format(prompt),
        prompt5.format(prompt)
    ]

    generated_texts = []

    for final_prompt in formatted_prompts:
        encoding = tokenizer(final_prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = encoding.input_ids.to(model.device)
        attention_mask = encoding.attention_mask.to(model.device)

        for _ in range(num_samples):  # 각 프롬프트당 num_samples개의 문장 생성
            with torch.no_grad():
                output = model.generate(
                    input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=30,  # `max_length` 대신 `max_new_tokens`
                    temperature=0.5,
                    top_p=0.8,
                    do_sample=True,
                    repetition_penalty=1.2
                )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
            generated_texts.append(generated_text)

    return generated_texts  # 50개의 문장 리스트 반환

In [18]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/My Drive/졸업논문/DATA/NSMC_0.02_증분대상100개.csv', encoding='utf-8-sig')

In [ ]:
df[[f'aug{i+1}' for i in range(50)]] = df['document'].apply(lambda x: pd.Series(generate_augmented_texts(x, num_samples=10)))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
df.to_csv('kogpt증분완료_0.02.csv', encoding='utf-8-sig')